In [6]:
import subprocess
import json
import pickle

DEFAULT_ATTRIBUTES = (
    'index',
    'uuid',
    'name',
    'timestamp',
    'memory.total',
    'memory.free',
    'memory.used',
    'utilization.gpu',
    'utilization.memory'
)

def get_gpu_info(nvidia_smi_path='nvidia-smi', keys=DEFAULT_ATTRIBUTES, no_units=True):
    nu_opt = '' if not no_units else ',nounits'
    cmd = '%s --query-gpu=%s --format=csv,noheader%s' % (nvidia_smi_path, ','.join(keys), nu_opt)
    output = subprocess.check_output(cmd, shell=True)
    lines = output.decode().split('\n')
    lines = [ line.strip() for line in lines if line.strip() != '' ]

    return [ { k: v for k, v in zip(keys, line.split(', ')) } for line in lines ]


import pprint
pprint.pprint(get_gpu_info())

[{'index': '0',
  'memory.free': '4781',
  'memory.total': '16160',
  'memory.used': '11379',
  'name': 'Tesla V100-SXM2-16GB',
  'timestamp': '2021/05/08 07:01:30.053',
  'utilization.gpu': '67',
  'utilization.memory': '39',
  'uuid': 'GPU-5271dac4-fa67-8890-3e2c-2d9a60803d6f'}]


# Imports

In [7]:
!pip install -q pydot
!pip install -q pydotplus
!apt-get install -q graphviz

import pandas as pd; pd.options.mode.chained_assignment = None;
import numpy as np; print(f"\t\t– NUMPY VERSION: {np.__version__}");
import scipy; print(f"\t\t– SCIPY VERSION: {scipy.__version__}");


from collections import Counter
from datetime import datetime
import multiprocessing
from glob import glob
import warnings
import requests
import IPython
import urllib
import zipfile
import pickle
import random
import shutil
import string
import math
import tqdm
import time
import gzip
import io
import os
import gc
import re

from matplotlib.colors import ListedColormap
import matplotlib.patches as patches
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

import seaborn as sns
from PIL import Image
import matplotlib; print(f"\t\t– MATPLOTLIB VERSION: {matplotlib.__version__}");

import PIL
import cv2
import ast

from torch.utils.data.sampler import SubsetRandomSampler, RandomSampler, SequentialSampler
from torch.optim.lr_scheduler import CosineAnnealingLR
# from warmup_scheduler import GradualWarmupScheduler  # https://github.com/ildoonet/pytorch-gradual-warmup-lr

import librosa, librosa.display
import soundfile as sf
from pathlib import Path

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
from typing import Optional, List

from tqdm import tqdm
from functools import partial
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GroupKFold
from transformers import get_linear_schedule_with_warmup

from albumentations import (
    Compose, OneOf, Normalize, Resize, RandomResizedCrop, RandomCrop, HorizontalFlip, VerticalFlip, 
    RandomBrightness, RandomContrast, RandomBrightnessContrast, Rotate, ShiftScaleRotate, Cutout, 
    IAAAdditiveGaussianNoise, Transpose, HueSaturationValue, CoarseDropout,OpticalDistortion,GridDistortion
    )
from albumentations.pytorch import ToTensorV2
from albumentations import ImageOnlyTransform
from torch.utils.data import DataLoader, Dataset

from torch.optim import Adam, SGD
from torch.cuda.amp import autocast, GradScaler
from sklearn.metrics import roc_auc_score

import timm
import torchvision.models as models
import pathlib
from contextlib import contextmanager

E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?

... OTHER IMPORTS STARTING ...


	VERSION INFORMATION
		– NUMPY VERSION: 1.19.5
		– SCIPY VERSION: 1.6.2
		– MATPLOTLIB VERSION: 3.4.1


In [8]:
# !pip install -q transformers
# !pip -q install timm
# !pip -q install git+https://github.com/ildoonet/pytorch-gradual-warmup-lr.git
# !pip -q install albumentations==0.5.2
# !pip install librosa
# !sudo apt-get install libsndfile1

# HELPER FUNCTIONS

In [11]:
def load_image_scaled(img_id, img_dir, img_size=512, load_style="tf"):
    """ Load An Image Using ID and Directory Path - Composes 4 Individual Images """
    def __load_with_tf(path, img_size=512):
        img = tf.io.read_file(path)
        img = tf.image.decode_png(img, channels=1)
        return tf.image.resize(img, (img_size, img_size))[..., 0]
    
    def __load_with_pil(path, img_size=512):
        img = Image.open(path)
        img = img.resize((img_size, img_size))
        return np.asarray(img)
    
    def __load_with_cv2(path, img_size=512):
        img = cv2.imread(path, 0)
        img = cv2.resize(img, (img_size, img_size))
        return img
        
    if load_style is "tf":
        load_fn = __load_with_tf
    elif load_style is "PIL":
        load_fn = __load_with_pil
    else:
        load_fn = __load_with_cv2
    
    return np.stack(
        [np.asarray(load_fn(os.path.join(img_dir, img_id+f"_{c}.png"), img_size)/255.) for c in ["red", "yellow", "blue"]], axis=2
    )


def decode_img(img, img_size=(224,224)):
    """TBD"""
    
    # convert the compressed string to a 3D uint8 tensor
    img = tf.image.decode_png(img, channels=1)

    # resize the image to the desired size
    return tf.cast(tf.image.resize(img, img_size), tf.uint8)


# image loader, using rgb only here
def load_RGBY_image(image_id, train_or_test='train', image_size=None):
    red = read_img(image_id, "red", train_or_test, image_size)
    green = read_img(image_id, "green", train_or_test, image_size)
    blue = read_img(image_id, "blue", train_or_test, image_size)
    #yellow = read_img(image_id, "yellow", train_or_test, image_size)
    stacked_images = np.transpose(np.array([red, green, blue]), (1,2,0))
    return stacked_images

# 
def read_img(image_id, color, train_or_test='train', image_size=None):
    filename = f'{ROOT}/{train_or_test}/{image_id}_{color}.png'
    assert os.path.exists(filename), f'not found {filename}'
    img = cv2.imread(filename, cv2.IMREAD_UNCHANGED)
    if image_size is not None:
        img = cv2.resize(img, (image_size, image_size))
    if img.max() > 255:
        img_max = img.max()
        img = (img/255).astype('uint8')
    return img

# make annotation helper called multi processes
def mk_ann(idx):
    image_id = df.iloc[idx].ID
    class_id = df.iloc[idx].Label
    anno = mk_mmdet_custom_data(image_id, class_id)
    img = load_RGBY_image(image_id, train_or_test)
    cv2.imwrite(f'{img_dir}/{image_id}.jpg', img)
    return anno, idx, image_id


def get_color_path_maps(color_dirs, tp_id_map):
    c_p_maps = [{k:[] for k in INT_2_STR.keys()} for _ in range(len(color_dirs))]
    color_d_paths = [
        [d_path for d_path in os.listdir(color_dir) if d_path.endswith("_256")] \
        for color_dir in color_dirs
    ]
    for c in tqdm(color_d_paths[0], total=len(color_d_paths[0])):
        
        # Get class stuff
        cls = c.split("_", 1)[1].rsplit("_",1)[0]
        cls_idx = STR_2_INT_LOWER[cls]
        
        # Get the relevant color directories
        c_dirs = [
            os.path.join(color_dir, c.replace("red", clr), "data", "train_tiles", cls) \
            for clr, color_dir in zip(["red", "green", "blue", "yellow"], color_dirs)
        ]

        # Update map
        for f_name in tqdm(os.listdir(c_dirs[0]), total=len(os.listdir(c_dirs[0]))):
            # get the relevant full paths
            full_paths = [os.path.join(c_dir, f_name.replace("red", clr)) for clr, c_dir in zip(["red", "green", "blue", "yellow"], c_dirs)]
            if tp_id_map==None:
                for c_p_map, full_path in zip(c_p_maps, full_paths):
                    c_p_map[cls_idx].append(full_path)
            elif (f_name.endswith(".png") and ("negative" in full_paths[0] or f_name.rsplit("_", 1)[0] in tp_id_map[cls_idx])):
                for c_p_map, full_path in zip(c_p_maps, full_paths):
                    c_p_map[cls_idx].append(full_path)
            else:
                for c_p_map, full_path in zip(c_p_maps, full_paths):
                    c_p_map[STR_2_INT["Negative"]].append(full_path)
    return [{k:sorted(v) for k,v in c_p_map.items()} for c_p_map in c_p_maps]


def get_tp_id_map(pkl_dir):
    """ TBD """
    # Capture all relevant paths
    pkl_paths = [
        os.path.join(pkl_dir, f_name) \
        for f_name in os.listdir(pkl_dir) \
        if f_name.endswith(".pkl")
    ]
    
    # REMOVE AFTER UPDATING CLASSBASED NOTEBOOK
    pkl_paths.append("/kaggle/input/tmp-intermediate-filaments-pkl-file/intermediate_filaments_tp_list.pkl")
    
    # Initialize
    tp_id_map = {}
    for path in pkl_paths:
        class_id = STR_2_INT_LOWER[path.rsplit("/", 1)[1].replace("_tp_list.pkl", "")]
        with open(path, "rb") as f:
            tp_id_map[class_id] = pickle.load(f)
    return tp_id_map

    
def plot_rgb(arr, figsize=(12,12)):
    """ Plot 3 Channel Microscopy Image """
    plt.figure(figsize=figsize)
    plt.title(f"RGB Composite Image", fontweight="bold")
    plt.imshow(arr)
    plt.axis(False)
    plt.show()    

    
def convert_rgby_to_rgb(arr):
    """ Convert a 4 channel (RGBY) image to a 3 channel RGB image.
    
    Advice From Competition Host/User: lnhtrang

    For annotation (by experts) and for the model, I guess we agree that individual 
    channels with full range px values are better. 
    In annotation, we toggled the channels. 
    For visualization purpose only, you can try blending the channels. 
    For example, 
        - red = red + yellow
        - green = green + yellow/2
        - blue=blue.
        
    Args:
        arr (numpy array): The RGBY, 4 channel numpy array for a given image
    
    Returns:
        RGB Image
    """
    
    rgb_arr = np.zeros_like(arr[..., :-1])
    rgb_arr[..., 0] = arr[..., 0]
    rgb_arr[..., 1] = arr[..., 1]+arr[..., 3]/2
    rgb_arr[..., 2] = arr[..., 2]
    
    return rgb_arr
    
    
def plot_ex(arr, figsize=(20,6), title=None, plot_merged=True, rgb_only=False):
    """ Plot 4 Channels Side by Side """
    if plot_merged and not rgb_only:
        n_images=5 
    elif plot_merged and rgb_only:
        n_images=4
    elif not plot_merged and rgb_only:
        n_images=4
    else:
        n_images=3
    plt.figure(figsize=figsize)
    if type(title) == str:
        plt.suptitle(title, fontsize=20, fontweight="bold")

    for i, c in enumerate(["Red Channel – Microtubles", "Green Channel – Protein of Interest", "Blue - Nucleus", "Yellow – Endoplasmic Reticulum"]):
        if not rgb_only:
            ch_arr = np.zeros_like(arr[..., :-1])        
        else:
            ch_arr = np.zeros_like(arr)
        if c in ["Red Channel – Microtubles", "Green Channel – Protein of Interest", "Blue - Nucleus"]:
            ch_arr[..., i] = arr[..., i]
        else:
            if rgb_only:
                continue
            ch_arr[..., 0] = arr[..., i]
            ch_arr[..., 1] = arr[..., i]
        plt.subplot(1,n_images,i+1)
        plt.title(f"{c.title()}", fontweight="bold")
        plt.imshow(ch_arr)
        plt.axis(False)
        
    if plot_merged:
        plt.subplot(1,n_images,n_images)
        
        if rgb_only:
            plt.title(f"Merged RGB", fontweight="bold")
            plt.imshow(arr)
        else:
            plt.title(f"Merged RGBY into RGB", fontweight="bold")
            plt.imshow(convert_rgby_to_rgb(arr))
        plt.axis(False)
        
    plt.tight_layout(rect=[0, 0.2, 1, 0.97])
    plt.show()
    
    
def flatten_list_of_lists(l_o_l):
    return [item for sublist in l_o_l for item in sublist]


def create_input_list(crp, cgp, cbp, cyp, shuffle=True, val_split=0.025):
    lbl_arr = flatten_list_of_lists([[k,]*len(v) for k, v in sorted(crp.items())])
    cr_arr = flatten_list_of_lists([v for k,v in sorted(crp.items())])
    cg_arr = flatten_list_of_lists([v for k,v in sorted(cgp.items())])
    cb_arr = flatten_list_of_lists([v for k,v in sorted(cbp.items())])
    cy_arr = flatten_list_of_lists([v for k,v in sorted(cyp.items())])
    
    if val_split is not None:
        val_lbl_arr = lbl_arr[:int(len(lbl_arr)*val_split)]
        lbl_arr = lbl_arr[int(len(lbl_arr)*val_split):]
        
        val_cr_arr = cr_arr[:int(len(cr_arr)*val_split)]
        cr_arr = cr_arr[int(len(cr_arr)*val_split):]
        
        val_cg_arr = cg_arr[:int(len(cg_arr)*val_split)]
        cg_arr = cg_arr[int(len(cg_arr)*val_split):]
        
        val_cb_arr = cb_arr[:int(len(cb_arr)*val_split)]
        cb_arr = cb_arr[int(len(cb_arr)*val_split):]

        val_cy_arr = cy_arr[:int(len(cy_arr)*val_split)]
        cy_arr = cy_arr[int(len(cy_arr)*val_split):]
        
    if shuffle:
        to_shuffle = list(zip(cr_arr, cg_arr, cb_arr, cy_arr, lbl_arr))
        random.shuffle(to_shuffle)
        cr_arr, cg_arr, cb_arr, cy_arr, lbl_arr = zip(*to_shuffle)
        
        if val_split is not None:
            val_to_shuffle = list(zip(val_cr_arr, val_cg_arr, val_cb_arr, val_cy_arr, val_lbl_arr))
            random.shuffle(val_to_shuffle)
            val_cr_arr, val_cg_arr, val_cb_arr, val_cy_arr, val_lbl_arr = zip(*val_to_shuffle)
    
    if val_split is None:
        return list(cr_arr), list(cg_arr), list(cb_arr), list(cy_arr), list(lbl_arr)
    else:
        return (list(cr_arr), list(cg_arr), list(cb_arr), list(cy_arr), list(lbl_arr)), \
               (list(val_cr_arr), list(val_cg_arr), list(val_cb_arr), list(val_cy_arr), list(val_lbl_arr))


def get_class_wts(single_ch_paths, n_classes=19, exclude_mitotic=True, multiplier=10, return_counts=False):
    """ TBD """
    # Get class counts
    class_counts = {c_idx:len(single_ch_paths[c_idx]) for c_idx in range(n_classes)}

    # Exclude mitotic spindle
    if exclude_mitotic:
        real_min_count = list(sorted(class_counts.values(), reverse=True))[-2]
    else:
        real_min_count = list(sorted(class_counts.values(), reverse=True))[-1]

    # Calculate weights
    class_wts = {k:min(1, multiplier*(real_min_count/v)) for k,v in class_counts.items()}

    if exclude_mitotic:
        # Manually adjust mitotic spindle to a more appropriate value
        class_wts[min(class_counts, key=class_counts.get)] = 1.0

    if return_counts:
        return class_wts, class_counts
    else:
        return class_wts

In [12]:
def rle_to_mask(rle_string, height, width):
    """ Convert RLE sttring into a binary mask 
    
    Args:
        rle_string (rle_string): Run length encoding containing 
            segmentation mask information
        height (int): Height of the original image the map comes from
        width (int): Width of the original image the map comes from
    
    Returns:
        Numpy array of the binary segmentation mask for a given cell
    """
    rows,cols = height,width
    rle_numbers = [int(num_string) for num_string in rle_string.split(' ')]
    rle_pairs = np.array(rle_numbers).reshape(-1,2)
    img = np.zeros(rows*cols,dtype=np.uint8)
    for index,length in rle_pairs:
        index -= 1
        img[index:index+length] = 255
    img = img.reshape(cols,rows)
    img = img.T
    return img

In [13]:
def read_img(image_path,image_size=256):
#     filename = f'{ROOT}/{train_or_test}/{image_id}_{color}.png'
#     assert os.path.exists(filename), f'not found {filename}'
    img = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
#     if image_size is not None:
#         img = cv2.resize(img, (image_size, image_size))
    if img.max() > 255:
        img_max = img.max()
        print('==============img_max')
        img = (img/255).astype('uint8')
    return img

In [14]:
def concat_label(cels_ids):
    cels_ids = str(cels_ids)
    cels_ids = eval(cels_ids)
    len_cels_ids = len(cels_ids)
    len_cels_ids_list = []
    for x in range(len_cels_ids):
        len_cels_ids_list.append(cels_ids[x].split(','))
#     print(len_cels_ids_list)
        
#     cels_ids = cels_ids.split(',')#cels_ids[0].split(',')
    labels = np.zeros(19, dtype="f")
    labels = []
    for i in len_cels_ids_list:
        i = i[0]
        i = int(i)
        labels.append(i)
#         labels[i] = 1
    return labels
        
def set_label(labe): 
    return list(set(labe))   

def how_len(yum):
    if len(yum) > 1:
        return 1
    else:
        return 0 

In [ ]:
# Fix Warmup Bug
from warmup_scheduler import GradualWarmupScheduler  

class GradualWarmupSchedulerV2(GradualWarmupScheduler):
    def __init__(self, optimizer, multiplier, total_epoch, after_scheduler=None):
        super(GradualWarmupSchedulerV2, self).__init__(optimizer, multiplier, total_epoch, after_scheduler)
    def get_lr(self):
        if self.last_epoch > self.total_epoch:
            if self.after_scheduler:
                if not self.finished:
                    self.after_scheduler.base_lrs = [base_lr * self.multiplier for base_lr in self.base_lrs]
                    self.finished = True
                return self.after_scheduler.get_lr()
            return [base_lr * self.multiplier for base_lr in self.base_lrs]
        if self.multiplier == 1.0:
            return [base_lr * (float(self.last_epoch) / self.total_epoch) for base_lr in self.base_lrs]
        else:
            return [base_lr * ((self.multiplier - 1.) * self.last_epoch / self.total_epoch + 1.) for base_lr in self.base_lrs]

# load csv

In [ ]:
def make_saved_path_not_pub(df):
    return 'train/' + df

def make_saved_path_pub(df):
    return 'publichpa_1024/' + df

def split_str(df):
    common_path = df.split('_')[0]
    return common_path

def find_label(label):
    return ('0' in label and '1' in label and  '2' in label)
    
def find_label(label):
    return ('0' in label and '1' in label and  '3' in label)

In [18]:
train_df_not_pub = pd.read_csv('train_df_not_pub.csv')
train_df_pub = pd.read_csv('train_df_pub.csv')

In [25]:
train_df_not_pub['saved_path'] = train_df_not_pub.common_ID.map(make_saved_path_not_pub)

In [28]:
train_df_pub['saved_path'] = train_df_pub.common_ID.map(make_saved_path_pub)

In [32]:
train_all_df = pd.concat([train_df_pub,train_df_not_pub]).reset_index(drop=True)

# Aug

In [82]:
# # ====================================================
# Transforms
# ====================================================
def get_transforms(*, data):  
    if data == 'train':
        return Compose([
#             ToGray(),
#             Resize(CFG.size, CFG.size),
            RandomResizedCrop(256, 256, scale=(0.85, 1.0)),
            HorizontalFlip(p=0.5),
            VerticalFlip(p=0.5),
            RandomBrightnessContrast(p=0.1, brightness_limit=(-0.1, 0.1), contrast_limit=(-0.1, 0.1)),
            HueSaturationValue(p=0.1, hue_shift_limit=0.1, sat_shift_limit=0.1, val_shift_limit=0.1),
            ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=5, border_mode=cv2.BORDER_CONSTANT, p=0.8),
            Cutout(p=0.5, max_h_size=16, max_w_size=16, fill_value=(0.), num_holes=50),
            Normalize(mean = (0.0680, 0.0583, 0.0566), std = (0.1464, 0.1249, 0.1612)
            ),
            ToTensorV2(),
        ])

    elif data == 'valid':
        return Compose([
#             Resize(CFG.size, CFG.size),
            Normalize(mean = (0.0680, 0.0583, 0.0566), std = (0.1464, 0.1249, 0.1612)
            ),
            ToTensorV2(),
        ])

# MODEL

In [24]:
class Efficientnet_b7(nn.Module):
    def __init__ (self):
        super().__init__()
        self.backbone = timm.create_model('tf_efficientnet_b7_ns', pretrained=True)
        in_ch = self.backbone.classifier.in_features
        self.backbone.classifier = nn.Linear(in_ch, 19)

    def forward(self, x):
        x = self.backbone(x)
        return x

In [25]:
def get_n_params(model):
    pp=0
    for p in list(model.parameters()):
        nn=1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    return pp
model = Efficientnet_b7()
    
para = get_n_params(model)/1000000
print(f'{para}M') # 14.9M (Ours) / 15M(Paper)

28.379715M


# LOSS

In [33]:
class FocalLoss_(nn.Module):
    def __init__(self, gamma=2):
        super().__init__()
        self.gamma = gamma

    def forward(self, input, target):
        if not (target.size() == input.size()):
            raise ValueError("Target size ({}) must be the same as input size ({})"
                             .format(target.size(), input.size()))

        max_val = (-input).clamp(min=0)
        loss = input - input * target + max_val + \
               ((-max_val).exp() + (-input - max_val).exp()).log()

        invprobs = F.logsigmoid(-input * (target * 2.0 - 1.0))
        loss = (invprobs * self.gamma).exp() * loss

        return loss.sum(dim=1).mean()
    
class FbetaLoss(nn.Module):
    def __init__(self, beta=1):
        super(FbetaLoss, self).__init__()
        self.small_value = 1e-6
        self.beta = beta

    def forward(self, logits, labels):
        beta = self.beta
        batch_size = logits.size()[0]
        p = F.sigmoid(logits)
        l = labels
        num_pos = torch.sum(p, 1) + self.small_value
        num_pos_hat = torch.sum(l, 1) + self.small_value
        tp = torch.sum(l * p, 1)
        precise = tp / num_pos
        recall = tp / num_pos_hat
        fs = (1 + beta * beta) * precise * recall / (beta * beta * precise + recall + self.small_value)
        loss = fs.sum() / batch_size
        return 1 - loss

class CombineLoss(nn.Module):
    def __init__(self):
        super(CombineLoss, self).__init__()
        self.fbeta_loss = FbetaLoss(beta=2)
        self.focal_loss = FocalLoss_()
        
    def forward(self, logits, labels):
        loss_beta = self.fbeta_loss(logits, labels)
        loss_focal = self.focal_loss(logits, labels)
        return 0.5 * loss_beta + 0.5 * loss_focal

In [34]:
#TTA
def get_trans(img, I):
    if I >= 4:
        img = img.transpose(2,3)
    if I % 4 == 0:
        return img
    elif I % 4 == 1:
        return img.flip(2)
    elif I % 4 == 2:
        return img.flip(3)
    elif I % 4 == 3:
        return img.flip(2).flip(3)

In [35]:
# ====================================================
# Helper functions
# ====================================================
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (remain %s)' % (asMinutes(s), asMinutes(rs))


def train_fn(train_loader, model, criterion, optimizer, epoch, scheduler, device):
    if CFG.device == 'GPU':
        scaler = GradScaler()
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    scores = AverageMeter()
    # switch to train mode
    model.train()
    start = end = time.time()
    global_step = 0
    for step, (images, labels) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)
        images = images.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        if CFG.device == 'GPU':
            with autocast():
                y_preds = model(images)
                
                # loss = lsep_loss_stable(y_preds, labels)
                loss = criterion(y_preds, labels)
                # record loss
                losses.update(loss.item(), batch_size)
                if CFG.gradient_accumulation_steps > 1:
                    loss = loss / CFG.gradient_accumulation_steps
                scaler.scale(loss).backward()
                grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), CFG.max_grad_norm)
                if (step + 1) % CFG.gradient_accumulation_steps == 0:
                    scaler.step(optimizer)
                    scaler.update()
                    optimizer.zero_grad()
                    global_step += 1

                # ############
                # if (step + 1) % 2000 == 0:
                #   scheduler.step()
                # ############
        
        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        if CFG.device == 'GPU':
            if step % CFG.print_freq == 0 or step == (len(train_loader)-1):
                print('Epoch: [{0}][{1}/{2}] '
                      'Data {data_time.val:.3f} ({data_time.avg:.3f}) '
                      'Elapsed {remain:s} '
                      'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                      'Grad: {grad_norm:.4f}  '
                      'LR: {lr:.6f}  '
                      .format(
                       epoch, step, len(train_loader), batch_time=batch_time,
                       data_time=data_time, loss=losses,
                       remain=timeSince(start, float(step+1)/len(train_loader)),
                       grad_norm=grad_norm,
                       lr=scheduler.get_lr()[0],
                       ))
    return losses.avg


def valid_fn(valid_loader, model, criterion, device, epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    scores = AverageMeter()
    # switch to evaluation mode
    model.eval()
    trues = []
    preds = []
    start = end = time.time()
    for step, (images, labels) in enumerate(valid_loader):
        # measure data loading time
        data_time.update(time.time() - end)
        images = images.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        # compute loss
        with torch.no_grad():
            probs = torch.zeros((images.shape[0], 19)).to(device)#(1,19)1_cellづつだから１だが、batchでも本来可能
            if epoch < 9:
                TTA_REPEATS = 1
            else:
                TTA_REPEATS = 4
            for I in range(TTA_REPEATS):
                output = model(get_trans(images, I))
                probs += torch.sigmoid(output)
            probs /= TTA_REPEATS
            y_preds = probs
        loss = criterion(y_preds, labels)
        # loss = lsep_loss_stable(y_preds, labels)
        losses.update(loss.item(), batch_size)
        # record accuracy
        trues.append(labels.to('cpu').numpy())
        preds.append(y_preds.to('cpu').numpy())
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        if CFG.device == 'GPU':
            if step % CFG.print_freq == 0 or step == (len(valid_loader)-1):
                print('EVAL: [{0}/{1}] '
                      'Data {data_time.val:.3f} ({data_time.avg:.3f}) '
                      'Elapsed {remain:s} '
                      'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                      .format(
                       step, len(valid_loader), batch_time=batch_time,
                       data_time=data_time, loss=losses,
                       remain=timeSince(start, float(step+1)/len(valid_loader)),
                       ))
        
    trues = np.concatenate(trues)
    predictions = np.concatenate(preds)
    return losses.avg, predictions, trues

# Dataset

In [39]:
train_all_df

,common_ID,saved_label,saved_path
0,1231_G1_1,"[0, 13]",publichpa_1024/1231_G1_1
1,264_A10_2,"[2, 13, 16]",publichpa_1024/264_A10_2
2,198_F5_1,"[0, 16]",publichpa_1024/198_F5_1
3,488_C4_1,"[0, 13]",publichpa_1024/488_C4_1
4,733_F7_1,"[0, 1]",publichpa_1024/733_F7_1
...,...,...,...
33915,ffd298f4-bbc7-11e8-b2bc-ac1f6b6435d0,"[16, 0, 4]",train/ffd298f4-bbc7-11e8-b2bc-ac1f6b6435d0
33916,ffd2b880-bba8-11e8-b2ba-ac1f6b6435d0,"[16, 0, 3]",train/ffd2b880-bba8-11e8-b2ba-ac1f6b6435d0
33917,ffe55eba-bbba-11e8-b2ba-ac1f6b6435d0,"[0, 5]",train/ffe55eba-bbba-11e8-b2ba-ac1f6b6435d0
33918,ffe61798-bbc3-11e8-b2bc-ac1f6b6435d0,"[12, 14, 7]",train/ffe61798-bbc3-11e8-b2bc-ac1f6b6435d0


In [47]:
#3色混ぜて出力
class HPADataset(data.Dataset):
    def __init__(
            self,
            df,
            transform=None):
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx: int):
        record = self.df.iloc[idx]
        dir_path = '/home/ubuntu/'
        path_img = record['saved_path']
        img = np.stack([
        cv2.imread(f'/home/ubuntu/{path_img}_red.png', 0),
        cv2.imread(f'/home/ubuntu/{path_img}_green.png', 0),
        cv2.imread(f'/home/ubuntu/{path_img}_blue.png', 0),
        ], axis=2)
        if img.max() > 255:#if uint16 
                img = cv2.convertScaleAbs(img, alpha=(255.0/65535.0))#uint8

        if self.transform:
            augmented = self.transform(image=img)
            img = augmented['image']

        labels = np.zeros(19, dtype="f")
        cels_ids = record['saved_label']#.split(',') 

        for i in cels_ids:
            i = int(i)
            labels[i] = 1

        # img = img.float()
        labels = torch.tensor(labels).float()
        return img,labels

# Visualization

In [50]:
# from matplotlib import pyplot as plt

# train_dataset = HPADataset(train_all_df, transform=get_transforms(data='train'))#get_transforms(data='train')

# for i in range(20000,20005,1):
#     image, label = train_dataset[i]
#     print(image.shape)
#     print(image.dtype)
#     print(image[0].max())
#     print(image[0].min())
#     plt.imshow(image[0])
#     plt.show()
#     plt.imshow(image[1])
#     plt.show()
#     plt.imshow(image[2])
#     plt.show()
# #     plt.imshow(image.cpu().numpy().transpose((1, 2, 0)))
#     print(image.transpose(0, 1).transpose(1,2).squeeze().shape)
#     plt.imshow(image.transpose(0, 1).transpose(1,2).squeeze())
# #     plt.imshow(image.squeeze().permute(1,2,0))
#     plt.title(f'label: {label}')
#     plt.show() 
#     plt.imshow(np.array(image).transpose((1,2,0)))
#     plt.show()

# Config

In [63]:
# ====================================================
# CFG
# ====================================================
OUTPUT_DIR =  'OUTPUT/exp27_hpa_train_cropmodel_eff5/'
TTA_REPEATS = 4

if not os.path.exists(f'/home/ubuntu/{OUTPUT_DIR}'):
    os.mkdir(f'/home/ubuntu/{OUTPUT_DIR}')

class CFG:
    debug=False
    trn_fold=[0]
    pretrain_weights =None#'/home/syuonez/OUTPUT/exp19_hpa_train_all_multi_1st_effnet5/tf_efficientnet_b5_fold0_epoch6.pth'
    device='GPU' 
    nprocs=1 
    print_freq=1000
    num_workers=4
    model_name='tf_efficientnet_b7'
    size=1024#<===
    scheduler='CosineAnnealingLR' 
    start_epcoh = 0
    freeze_epo = 0
    warmup_epo = 2
    cosine_epo = 10
    epochs= freeze_epo + warmup_epo + cosine_epo
    T_max=10
    warmup_epo = 2
    T_0=5
    lr=1e-4
    min_lr=1e-6
    batch_size=2
    weight_decay=1e-6
    gradient_accumulation_steps=1
    max_grad_norm=1000
    seed=42
    target_cols=['0', '1', '2','3', '4', '5', '6', '7', '8', '9','10', '11', '12', '13', '14', '15', '16', '17', '18']               
    n_fold=5
    train=True

In [64]:
# ====================================================
# Utils
# ====================================================
def get_score(y_true, y_pred):
    scores = []
    for i in range(y_true.shape[1]):
        try:
            score = roc_auc_score(y_true[:,i], y_pred[:,i])
            scores.append(score)
        except ValueError:
            pass
        
    avg_score = np.mean(scores)

    return avg_score, scores


@contextmanager
def timer(name):
    t0 = time.time()
    LOGGER.info(f'[{name}] start')
    yield
    LOGGER.info(f'[{name}] done in {time.time() - t0:.0f} s.')


def init_logger(log_file=OUTPUT_DIR+'train.log'):
    from logging import getLogger, INFO, FileHandler,  Formatter,  StreamHandler
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=log_file)
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = init_logger()


def seed_torch(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_torch(seed=CFG.seed)

# LOOP

In [87]:

# ====================================================
# Train loop
# ====================================================
def train_loop(folds, fold):
   
    if CFG.device == 'GPU':
        LOGGER.info(f"========== fold: {fold} training ==========")

    trn_idx = folds[folds['fold'] != fold].index
    val_idx = folds[folds['fold'] == fold].index
    train_folds = folds.loc[trn_idx].reset_index(drop=True)
    valid_folds = folds.loc[val_idx].reset_index(drop=True)
    
#     if CFG.debug:
#         train_folds = train_folds.sample(n=200, random_state=CFG.seed).reset_index(drop=True)
#     train_folds = train_folds.sample(n=100)
#     valid_folds = valid_folds.sample(n=100)
    train_dataset = HPADataset(train_folds, transform=get_transforms(data='train'))
    valid_dataset = HPADataset(valid_folds, transform=get_transforms(data='valid'))

    valid_labels = valid_folds['saved_label'].values

    
    train_loader = DataLoader(train_dataset, 
                              batch_size=CFG.batch_size, 
                              shuffle=True, 
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=True)
    valid_loader = DataLoader(valid_dataset, 
                              batch_size=CFG.batch_size * 2, 
                              shuffle=False, 
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=False)

    valid_labels_onehot_all = []
    for cel_labels_valid in valid_labels:
        valid_labels_onehot = np.zeros(19, dtype="f")
        cels_ids_ = eval(cel_labels_valid)#cel_labels_valid.split(',')
        for u in cels_ids_:
            t = int(u)
            valid_labels_onehot[t] = 1
        valid_labels_onehot_all.append(valid_labels_onehot)
    valid_labels_onehot_all = np.asarray(valid_labels_onehot_all)

    # ====================================================
    # model & optimizer & scheduler
    # ====================================================
    model = Efficientnet_b5()#SErexnext50()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    #####再学習#####
    if CFG.pretrain_weights:
        print("---------------------loading pretrain weights")
        checkpoint = torch.load(CFG.pretrain_weights)
        model.load_state_dict(checkpoint['model_state_dict'])
    model.to(device)
    #####再学習#####

#     optimizer = Adam(model.parameters(), lr=CFG.lr, weight_decay=CFG.weight_decay, amsgrad=False)
    optimizer = SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=1e-5)
#     optimizer = AdamP(model.parameters(), lr=CFG.lr, weight_decay=CFG.weight_decay)
#     optimizer = optim.RAdam(model.parameters(),lr= CFG.lr,betas=(0.9, 0.999),eps=1e-8,weight_decay=0,)

    scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, CFG.cosine_epo)
    scheduler = GradualWarmupSchedulerV2(optimizer, multiplier=10, total_epoch=CFG.warmup_epo, after_scheduler=scheduler_cosine)
    # scheduler = MultiStepLR(optimizer, milestones=[200, 350], gamma=0.5)
    # scheduler = get_scheduler(optimizer)
    
    # ====================================================
    # loop
    # ====================================================
    criterion = CombineLoss()#FocalLoss_()#CombineLoss()#CombineLoss()#ResampleLoss()#CombineLoss()

    best_score = 0.
    best_loss = np.inf
    
    ####再学習#####
    if CFG.pretrain_weights:
        print("---------------------loading pretrain weights")
        # model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        for state in optimizer.state.values():
            for k, v in state.items():
                if isinstance(v, torch.Tensor):
                    state[k] = v.to('cuda')
    ##############
    
    
    ####再学習#####
    current_epoch= 0
    if CFG.pretrain_weights:
        print('=>skip_current_epoch')
        current_epoch = checkpoint['epoch'] + 1 #train中に止まると仮定　epoch5中　(checkpoint['epoch']==4) + 1
        print('current_epoch',current_epoch)
        CFG.start_epcoh = current_epoch 
        print(f'Config.start_epcoh={CFG.start_epcoh}=======>Config.epochs={CFG.epochs}')
        for _ in range(current_epoch):#lrをcurrent_epochにする為
            scheduler.step()
 #===============================================================================   
    for epoch in range(CFG.start_epcoh, CFG.epochs):
        
        start_time = time.time()
        # train
        avg_loss = train_fn(train_loader, model, criterion, optimizer, epoch, scheduler, device)

        # eval
        avg_val_loss, preds, _ = valid_fn(valid_loader, model, criterion, device,epoch)

        #warmup
        scheduler.step()#epoch5でsave scheはepoch6状態

        #scoring
        score, scores = get_score(valid_labels_onehot_all, preds)

        elapsed = time.time() - start_time

        if CFG.device == 'GPU':
            LOGGER.info(f'Epoch {epoch} - avg_train_loss: {avg_loss:.4f}  avg_val_loss: {avg_val_loss:.4f}  time: {elapsed:.0f}s')
            LOGGER.info(f'Epoch {epoch} - Score: {score:.4f}  Scores: {np.round(scores, decimals=4)}')

        #save
        LOGGER.info(f'Epoch {epoch} - Save Best Score: {best_score:.4f} Model')
        torch.save({'model_state_dict': model.state_dict(), 
                          'preds': preds,
                          'epoch':epoch,
                          'optimizer_state_dict': optimizer.state_dict(),
                          'scheduler_state_dict': scheduler.state_dict()},
                           OUTPUT_DIR+f'{CFG.model_name}_fold{fold}_epoch{epoch}.pth')
        if score > best_score:
            best_score = score
            if CFG.device == 'GPU':
                LOGGER.info(f'Epoch {epoch} - Save Best Score: {best_score:.4f} Model')
                torch.save({'model_state_dict': model.state_dict(), 
                          'preds': preds,
                          'epoch':epoch,
                          'optimizer_state_dict': optimizer.state_dict(),
                          'scheduler_state_dict': scheduler.state_dict()},
                           OUTPUT_DIR+f'{CFG.model_name}_fold{fold}_best_score.pth')

        if avg_val_loss < best_loss:
            best_loss = avg_val_loss
            if CFG.device == 'GPU':
                LOGGER.info(f'Epoch {epoch} - Save Best Loss: {best_loss:.4f} Model')
                torch.save({'model_state_dict': model.state_dict(), 
                          'preds': preds,
                          'epoch':epoch,
                          'optimizer_state_dict': optimizer.state_dict(),
                          'scheduler_state_dict': scheduler.state_dict()},
                           OUTPUT_DIR+f'{CFG.model_name}_fold{fold}_best_loss.pth')
#==========================================================================================
            
    
    if CFG.nprocs != 8:
        check_point = torch.load(OUTPUT_DIR+f'{CFG.model_name}_fold{fold}_best_score.pth')
        row_len = len(check_point['preds'])
        oof_df = pd.DataFrame(np.zeros((row_len,19)),columns = CFG.target_cols)
        oof_df[CFG.target_cols] = check_point['preds']

    return oof_df,valid_labels_onehot_all

# main

In [89]:
# ====================================================
# main
# ====================================================
def main():
    """
    Prepare: 1.train  2.folds
    """
    def get_result(result_df,labels):#labels arr
        preds = result_df.values
#         labels = fold_oof[CFG.target_cols].values
        score, scores = get_score(labels, preds)
        LOGGER.info(f'Score: {score:<.4f}  Scores: {np.round(scores, decimals=4)}')
    
    if CFG.train:
        oof_label_all = []
        for cel_labels in train_all_df.saved_label.values:
            oof_labels_onehot = np.zeros(19, dtype="f")
            cels_ids = eval(cel_labels)
            for i in cels_ids:
                i = int(i)
                oof_labels_onehot[i] = 1
            oof_label_all.append(oof_labels_onehot)
        oof_label_all = np.asarray(oof_label_all)

        oof_df = pd.DataFrame()
        for fold in range(CFG.n_fold):
            if fold in CFG.trn_fold:
                _oof_df,fold_oof = train_loop(train_all_df, fold)
                oof_df = pd.concat([oof_df, _oof_df])
                LOGGER.info(f"========== fold: {fold} result ==========")
                get_result(_oof_df,fold_oof)
                if len(CFG.trn_fold) == 1:
                    print(f'======>saveing.....fold{fold}_oof_df.csv')
                    oof_df.to_csv(OUTPUT_DIR+f'fold{fold}_oof_df.csv', index=False)
                    
                    
        if CFG.nprocs != 8:
            # CV result
            LOGGER.info(f"========== CV ==========")
            get_result(oof_df,oof_label_all)
            # save result
            print(f'======>saveing.....{OUTPUT_DIR}all_fold_oof_df.csv')
            oof_df.to_csv(OUTPUT_DIR+'all_fold_oof_df.csv', index=False)

In [ ]:
if __name__ == '__main__':#exp19
    main()

========== fold: 0 training ==========
========== fold: 0 training ==========
/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch: [0][0/16559] Data 0.546 (0.546) Elapsed 0m 0s (remain 233m 17s) Loss: 1.9965(1.9965) Grad: 166244.1562  LR: 0.001000  
Epoch: [0][1000/16559] Data 0.000 (0.001) Elapsed 3m 42s (remain 57m 45s) Loss: 1.7692(1.8738) Grad: 149388.6875  LR: 0.001000  
Epoch: [0][2000/16559] Data 0.000 (0.000) Elapsed 7m 25s (remain 53m 57s) Loss: 1.5328(1.7596) Grad: 86461.0234  LR: 0.001000  
Epoch: [0][3000/16559] Data 0.000 (0.000) Elapsed 11m 6s (remain 50m 11s) Loss: 1.3486(1.6532) Grad: 84596.8672  LR: 0.001000  


In [52]:
exp19 = pd.read_csv('OUTPUT/exp19_hpa_train_all_multi_1st_effnet5/all_fold_oof_df.csv')

In [53]:
exp19

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,0.452583,0.065082,0.417432,0.121861,0.032187,0.269144,0.018252,0.132344,0.048919,0.105704,0.006168,0.006493,0.301697,0.161969,0.077482,0.057907,0.401986,0.093884,0.003126
1,0.454949,0.088162,0.334720,0.181567,0.012136,0.311893,0.013568,0.149284,0.068272,0.085308,0.017129,0.008582,0.467314,0.092692,0.079637,0.070031,0.218545,0.086124,0.011864
2,0.487202,0.069489,0.204527,0.057138,0.030402,0.130926,0.009681,0.079832,0.055931,0.198096,0.032273,0.037710,0.405196,0.135363,0.107625,0.072113,0.270291,0.191561,0.011221
3,0.454523,0.043706,0.190375,0.016845,0.040466,0.313066,0.015094,0.053603,0.013419,0.072310,0.001398,0.004281,0.186313,0.032767,0.047692,0.014605,0.518120,0.061848,0.000529
4,0.517837,0.142062,0.294394,0.091637,0.009651,0.272183,0.008066,0.111612,0.064060,0.105985,0.008542,0.004384,0.438428,0.102441,0.073953,0.076477,0.174567,0.111220,0.007209
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59806,0.332458,0.122087,0.194189,0.452079,0.250288,0.371924,0.010424,0.183149,0.131522,0.250703,0.025751,0.006302,0.208870,0.270853,0.138432,0.076697,0.251885,0.176746,0.012299
59807,0.473109,0.100693,0.173211,0.183406,0.076741,0.193342,0.026596,0.191492,0.180238,0.311742,0.067532,0.006776,0.246767,0.289809,0.165872,0.055248,0.334935,0.069367,0.010210
59808,0.405866,0.112408,0.223267,0.232118,0.103992,0.265668,0.018285,0.274021,0.164278,0.330791,0.026996,0.010769,0.310258,0.401818,0.137493,0.141468,0.273663,0.202106,0.017230
59809,0.201377,0.132124,0.118362,0.357336,0.175493,0.244609,0.053917,0.212638,0.171955,0.321731,0.073186,0.006951,0.313417,0.316121,0.082088,0.153788,0.425528,0.224824,0.023224
